<a href="https://colab.research.google.com/github/avinash064/Avinash_kashyap_21064/blob/main/21064_AVINASHkashyap_nlpassignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### NLP Assignment-2

Load data

In [ ]:
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/train_split.csv
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/test_split.csv

--2024-10-17 14:47:56--  https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/train_split.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 144474 (141K) [text/plain]
Saving to: ‘train_split.csv’

train_split.csv     100%[===================>] 141.09K  --.-KB/s    in 0.03s   

2024-10-17 14:47:56 (4.37 MB/s) - ‘train_split.csv’ saved [144474/144474]

--2024-10-17 14:47:56--  https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/test_split.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35259 (34K) 

In [6]:
train_data.isnull().sum()

emotion_columns = ['Joy', 'Fear', 'Anger', 'Sadness', 'Surprise']

In [ ]:
train_data.columns

Index(['text', 'Joy', 'Fear', 'Anger', 'Sadness', 'Surprise'], dtype='object')

###Evaluation
An example of evaluation:

In [97]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from collections import Counter
from nltk.corpus import stopwords

# Download the NLTK tokenizer
nltk.download('punkt')

# Load the dataset
train_data_url = 'https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/train_split.csv'
train_data = pd.read_csv(train_data_url)
test_data_url = 'https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/test_split.csv'
test_data = pd.read_csv(test_data_url)
# Assuming the dataset has a 'text' column
train_texts = train_data['text'].dropna().tolist()

# Tokenize the text data (convert to lowercase and split into words)
tokenized_sentences = [word_tokenize(text.lower()) for text in train_texts]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [98]:
nltk.download('stopwords')
nltk_stopwords = set(stopwords.words('english'))

# Function to preprocess and remove punctuation and stopwords
def preprocess_text(text):

    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ''.join([ch for ch in text if not ch.isdigit()]) # Removing numbers
    words = text.split()      # Split the text into words
    words = [word for word in words if word not in nltk_stopwords]

    return ' '.join(words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [102]:
import string

In [104]:
X_train = train_data.iloc[:, 0]
y_train = train_data.iloc[:, 1:]

X_train_cleaned = X_train.apply(preprocess_text)

print("\nCleaned X (features - text column after preprocessing):")
display(X_train_cleaned.head())

# Convert X_train_cleaned (Pandas Series) into a list of lists for Gensim
train_texts = X_train_cleaned.tolist()

# Display labels
print("\n y (target labels - emotion columns):")
display(y_train.head())


Cleaned X (features - text column after preprocessing):


,text
0,light would come way vehicle would seem vanish...
1,well birthday january
2,tears eyes much mind deep sadness heart
3,eyes scanned quickly every room every corner p...
4,felt afterglow late afternoon sun cool zephyr ...



 y (target labels - emotion columns):


,Joy,Fear,Anger,Sadness,Surprise
0,0,1,0,0,1
1,0,0,0,0,0
2,0,1,0,1,0
3,0,1,1,0,0
4,1,0,0,0,0


In [83]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, embeddings, labels):
        """
        Args:
            embeddings (numpy.ndarray or torch.Tensor): Preprocessed word embeddings for sentences.
            labels (numpy.ndarray or torch.Tensor): Corresponding labels (multi-output) for each sentence.
        """
        self.embeddings = torch.tensor(embeddings, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]


In [66]:
from sklearn.preprocessing import MultiLabelBinarizer

In [105]:

train_labels = train_data[['Joy', 'Fear', 'Anger', 'Sadness', 'Surprise']].values.tolist()

# Tokenize the text data (convert to lowercase and split into words)
tokenized_sentences = [word_tokenize(text.lower()) for text in train_texts]


In [106]:
print(tokenized_sentences)

[['light', 'would', 'come', 'way', 'vehicle', 'would', 'seem', 'vanish', 'side', 'heading', 'way'], ['well', 'birthday', 'january'], ['tears', 'eyes', 'much', 'mind', 'deep', 'sadness', 'heart'], ['eyes', 'scanned', 'quickly', 'every', 'room', 'every', 'corner', 'passed', 'fist', 'met', 'peeling', 'paint', 'closed', 'door'], ['felt', 'afterglow', 'late', 'afternoon', 'sun', 'cool', 'zephyr', 'skin', 'time', 'stood', 'outside', 'main', 'gates', 'university', 'saw', 'car', 'making', 'turn'], ['whatever', 'stuffing', 'head', 'yesterday', 'moved', 'chest'], ['point', 'decided', 'consult', 'woman', 'front', 'desk', 'information'], ['fucking', 'breaking', 'point'], ['could', 'wrong', 'though'], ['could', 'slaughter', 'turned', 'wheel'], ['bit', 'lower', 'lip', 'trying', 'hard', 'force', 'back', 'another', 'lump', 'throat'], ['mr', 'jackson', 'called', 'told', 'call', 'told', 'ask', 'didnt', 'call', 'thats', 'calling', 'broke', 'giggling', 'rushed', 'cover', 'mouth', 'stealing', 'embarrassed'

In [107]:
print(train_labels)

[[0, 1, 0, 0, 1], [0, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 1], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 1], [0, 1, 0, 1, 0], [0, 1, 0, 1, 1], [0, 1, 0, 0, 1], [0, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1], [0, 0, 0, 0, 0], [0, 1, 0, 1, 1], [0, 1, 0, 0, 1], [0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 1], [0, 1, 0, 1, 1], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 1], [0, 1, 0, 0, 1], [1, 0, 0, 0, 0], [0, 1, 1, 0, 1], [0, 0, 0, 0, 0], [0, 0, 1, 0, 1], [0, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 0, 0], [1, 0, 0, 0, 

In [108]:
print(train_texts)

['light would come way vehicle would seem vanish side heading way', 'well birthday january', 'tears eyes much mind deep sadness heart', 'eyes scanned quickly every room every corner passed fist met peeling paint closed door', 'felt afterglow late afternoon sun cool zephyr skin time stood outside main gates university saw car making turn', 'whatever stuffing head yesterday moved chest', 'point decided consult woman front desk information', 'fucking breaking point', 'could wrong though', 'could slaughter turned wheel', 'bit lower lip trying hard force back another lump throat', 'mr jackson called told call told ask didnt call thats calling broke giggling rushed cover mouth stealing embarrassed glance nurses watching complete awe', 'didnt legs keep hills fell back', 'messed shoulder pretty well damn near broke several fingers', 'fell knees body shaking sorrow held since woke', 'closest analogy could come', 'feel like ted sad sap', 'next day drain clogged', 'know true glasses gone years kn

In [111]:
from collections import defaultdict

# Creating word to index mapping
word_to_idx = {}
idx_to_word = {}
index = 0

for sentence in tokenized_sentences:
    for word in sentence:
        if word not in word_to_idx:
            word_to_idx[word] = index
            idx_to_word[index] = word
            index += 1

# Vocabulary size
vocab_size = len(word_to_idx)

print(f"\nVocabulary size: {vocab_size}")



Vocabulary size: 4009


In [112]:
# Build vocabulary from tokenized sentences
vocab = set([word for sentence in tokenized_sentences for word in sentence])
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}
vocab_sizes = len(vocab)

# Maximum sequence length and embedding size
MAX_SEQUENCE_LENGTH = 128
EMBEDDING_SIZE = 100

# Padding function
def pad_or_truncate(sentence, max_len=MAX_SEQUENCE_LENGTH):
    if len(sentence) > max_len:
        return sentence[:max_len]
    else:
        return sentence + [0] * (max_len - len(sentence))


class Word2Vec(nn.Module):
    def __init__(self, vocab_size, embed_size):
        super(Word2Vec, self).__init__()
        self.in_embed = nn.Embedding(vocab_size, embed_size)
        self.out_embed = nn.Embedding(vocab_size, embed_size)

    def forward(self, target_word, context_word, negative_samples):
        target_embedding = self.in_embed(target_word)
        context_embedding = self.out_embed(context_word)
        neg_sample_embedding = self.out_embed(negative_samples)
        pos_score = torch.mul(target_embedding, context_embedding).sum(dim=1)
        pos_score = torch.log(torch.sigmoid(pos_score))
        neg_score = torch.bmm(neg_sample_embedding, target_embedding.unsqueeze(2)).squeeze()
        neg_score = torch.log(torch.sigmoid(-neg_score)).sum(dim=1)
        return - (pos_score + neg_score).mean()

# Create Skip-Gram pairs for Word2Vec training
def generate_skipgram_pairs(tokenized_sentences, window_size=2):
    skipgram_pairs = []
    for sentence in tokenized_sentences:
        for i, word in enumerate(sentence):
            target_word = word_to_idx[word]
            context_range = list(range(max(0, i - window_size), min(len(sentence), i + window_size + 1)))
            context_range.remove(i)  # Remove the target word itself
            for j in context_range:
                context_word = word_to_idx[sentence[j]]
                skipgram_pairs.append((target_word, context_word))
    return skipgram_pairs

# Create skip-gram pairs
skipgram_pairs = generate_skipgram_pairs(tokenized_sentences, window_size=2)

# Function to generate negative samples
def get_negative_samples(batch_size, vocab_size, neg_sample_size=5):
    neg_samples = torch.randint(0, vocab_size, (batch_size, neg_sample_size), dtype=torch.long)
    return neg_samples

In [113]:
print(vocab_sizes)

4009


In [114]:
embedding_dim = 100

# Randomly initialize the embedding matrix
embedding_matrix = np.random.randn(vocab_size, embedding_dim) * 0.01


In [110]:
def train_word2vec(epochs=10, batch_size=64):
    model = Word2Vec(vocab_size, EMBEDDING_SIZE)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    for epoch in range(epochs):
        total_loss = 0
        random.shuffle(skipgram_pairs)
        for i in range(0, len(skipgram_pairs), batch_size):
            batch_pairs = skipgram_pairs[i:i + batch_size]
            if len(batch_pairs) < batch_size:
                continue
            target_words = torch.tensor([pair[0] for pair in batch_pairs], dtype=torch.long)
            context_words = torch.tensor([pair[1] for pair in batch_pairs], dtype=torch.long)
            negative_samples = get_negative_samples(batch_size, vocab_size)
            optimizer.zero_grad()
            loss = model(target_words, context_words, negative_samples)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {total_loss:.4f}")
    return model

# Train Word2Vec model
word2vec_model = train_word2vec()

# Convert tokenized sentences to word embeddings using the trained Word2Vec model
def sentence_to_embedding(sentence, word2vec_model):
    word_indices = [word_to_idx.get(word, 0) for word in sentence]  # Convert words to indices
    padded_sentence = pad_or_truncate(word_indices)  # Pad or truncate the sentence
    embeddings = word2vec_model.in_embed(torch.tensor(padded_sentence))  # Get embeddings
    return embeddings.detach().numpy()



Epoch [1/10], Loss: 14338.9597
Epoch [2/10], Loss: 13763.5901
Epoch [3/10], Loss: 13075.3592
Epoch [4/10], Loss: 12572.0549
Epoch [5/10], Loss: 11952.9821
Epoch [6/10], Loss: 11440.8109
Epoch [7/10], Loss: 10922.2438
Epoch [8/10], Loss: 10451.7085
Epoch [9/10], Loss: 10005.1557
Epoch [10/10], Loss: 9576.6148


In [115]:
# Convert all tokenized sentences to embeddings
sentence_embeddings_1 = [sentence_to_embedding(sentence, word2vec_model) for sentence in tokenized_sentences]
sentence_embeddings_1 = np.array(sentence_embeddings_1)  # This step will speed up the conversion process
print(sentence_embeddings_1)

[[[-1.66258430e+00  1.96950927e-01  5.35553060e-02 ...  4.25484180e-01
   -1.12152934e+00 -3.27906162e-01]
  [-3.66890371e-01 -2.68202275e-01 -1.54482707e-01 ... -2.24994197e-01
   -3.60156983e-01  1.33167645e-02]
  [-2.19266251e-01 -2.27203563e-01  2.54747476e-02 ...  1.30458847e-01
   -5.70153177e-01  1.95115677e-03]
  ...
  [ 1.93921709e+00  3.60577881e-01  8.83155882e-01 ...  1.41020346e+00
    1.17531908e+00 -5.98132074e-01]
  [ 1.93921709e+00  3.60577881e-01  8.83155882e-01 ...  1.41020346e+00
    1.17531908e+00 -5.98132074e-01]
  [ 1.93921709e+00  3.60577881e-01  8.83155882e-01 ...  1.41020346e+00
    1.17531908e+00 -5.98132074e-01]]

 [[-5.01791596e-01  1.30440474e-01  1.27178192e+00 ...  6.89557195e-01
    7.45034873e-01  7.26435959e-01]
  [ 1.84393215e+00 -9.84452188e-01  4.05787677e-01 ...  9.96062577e-01
    3.91788602e-01  3.35023880e-01]
  [-1.73264779e-02  1.14375427e-01  2.09451246e+00 ... -9.12587047e-01
   -9.47644234e-01 -7.48885036e-01]
  ...
  [ 1.93921709e+00  3.6

In [116]:
def create_sentence_embeddings(sentences):
    embeddings = []
    for sentence in sentences:
        word_embeddings = [word2vec_model.in_embed(torch.tensor(word_to_idx[word])) for word in sentence if word in word_to_idx]
        if word_embeddings:
            sentence_embedding = torch.mean(torch.stack(word_embeddings), dim=0)  # Average word embeddings
            embeddings.append(sentence_embedding)
        else:
            embeddings.append(torch.zeros(EMBEDDING_SIZE))  # For empty sentences
    return torch.stack(embeddings)

# Create sentence embeddings
sentence_embeddings_2 = create_sentence_embeddings(tokenized_sentences)

# Prepare labels
train_labels_tensor = torch.tensor(train_labels, dtype=torch.float32)

# Dataset class for loading preprocessed embeddings
class EmbeddingDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = torch.tensor(embeddings, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)  # Float for multi-labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]

# Create DataLoader
train_dataset = EmbeddingDataset(sentence_embeddings_2, train_labels_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Neural Network Class for FFNN
class FFNN(nn.Module):
    def __init__(self, embed_size, hidden_size, output_size):
        super(FFNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(embed_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
            nn.Sigmoid()  # Sigmoid for multi-label output
        )

    def forward(self, x):
        return self.model(x)

# Neural Network Class for RNN
class RNN(nn.Module):
    def __init__(self, embed_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add sequence dimension
        output, _ = self.rnn(x)
        output = self.fc(output[:, -1, :])  # Get last time step
        return torch.sigmoid(output)

# Neural Network Class for LSTM
class LSTM(nn.Module):
    def __init__(self, embed_size, hidden_size, output_size, num_layers=1):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.unsqueeze(1)
        output, _ = self.lstm(x)
        output = self.fc(output[:, -1, :])  # Get last time step
        return torch.sigmoid(output)

# Training Function
def train_neural_model(model_class, embed_size, hidden_size, output_size, epochs=10):
    model = model_class(embed_size, hidden_size, output_size)

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.BCELoss()  # For multi-label classification

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch_embeds, batch_labels in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_embeds)

            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {total_loss:.4f}")
        return model


print("Training FFNN...")
FNN = train_neural_model(FFNN, EMBEDDING_SIZE, 64, 5, epochs=10)

print("Training RNN...")
RNN = train_neural_model(RNN, EMBEDDING_SIZE, 64, 5, epochs=10)

print("Training LSTM...")
LSTM = train_neural_model(LSTM, EMBEDDING_SIZE, 64, 5, epochs=10)

<ipython-input-116-0be2036cbe19>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.embeddings = torch.tensor(embeddings, dtype=torch.float32)
<ipython-input-116-0be2036cbe19>:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels, dtype=torch.float32)  # Float for multi-labels


Training FFNN...
Epoch [1/10], Loss: 16.8452
Training RNN...
Epoch [1/10], Loss: 16.5570
Training LSTM...
Epoch [1/10], Loss: 17.0333


In [117]:
print(FNN)

FFNN(
  (model): Sequential(
    (0): Linear(in_features=100, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=5, bias=True)
    (3): Sigmoid()
  )
)


In [118]:
# Load the test dataset
test_data_url = 'https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/test_split.csv'
test_data = pd.read_csv(test_data_url)

# Assuming the dataset has a 'text' column and the labels are in multiple columns
test_texts = test_data['text'].dropna().tolist()
test_labels = test_data[['Joy', 'Fear', 'Anger', 'Sadness', 'Surprise']].dropna().values.tolist()

# Tokenize the text data (convert to lowercase and split into words)
test_tokenized_sentences = [word_tokenize(text.lower()) for text in test_texts]

# Create sentence embeddings for the test set
test_sentence_embeddings = create_sentence_embeddings(test_tokenized_sentences)

# Prepare test labels
test_labels_tensor = torch.tensor(test_labels, dtype=torch.float32)

# Create DataLoader for test dataset
test_dataset = EmbeddingDataset(test_sentence_embeddings, test_labels_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch_embeds, batch_labels in test_loader:
            outputs = model(batch_embeds)
            preds = (outputs > 0.5).float()  # Binarize outputs
            all_preds.append(preds)
            all_labels.append(batch_labels)

    # Concatenate all predictions and labels
    all_preds = torch.cat(all_preds).numpy()
    all_labels = torch.cat(all_labels).numpy()

    # Calculate F1 Macro Score
    f1_macro = f1_score(all_labels, all_preds, average='macro')
    return f1_macro

# Evaluate each model on the test dataset


<ipython-input-116-0be2036cbe19>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.embeddings = torch.tensor(embeddings, dtype=torch.float32)
<ipython-input-116-0be2036cbe19>:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels, dtype=torch.float32)  # Float for multi-labels


In [119]:
print("Evaluating FFNN...")
f1_ffnn = evaluate_model(FNN, test_loader)
print(f"FFNN F1 Macro Score: {f1_ffnn:.4f}")

print("Evaluating RNN...")
f1_rnn = evaluate_model(RNN, test_loader)
print(f"RNN F1 Macro Score: {f1_rnn:.4f}")

print("Evaluating LSTM...")
f1_lstm = evaluate_model(LSTM, test_loader)
print(f"LSTM F1 Macro Score: {f1_lstm:.4f}")


Evaluating FFNN...
FFNN F1 Macro Score: 0.1790
Evaluating RNN...
RNN F1 Macro Score: 0.1884
Evaluating LSTM...
LSTM F1 Macro Score: 0.1947


In [129]:
import torch


In [128]:
# Saving model state dictionary
torch.save(LSTM.state_dict(), 'LSTM_state.pth')
